In [47]:
import json
from openai import OpenAI
from search_tools import create_search_tools 

openai_client = OpenAI()
evidently_search_tools = create_search_tools()

In [13]:
evidently_search_tools.search('dashboards')[:2]

[{'title': 'Add dashboard panels (API)',
  'description': 'How to design your Dashboard with custom Panels.',
  'content': {'matches': ['...oard_add_panels_ui).\n\n## **Dashboard** Management\n\n<Check>\n  **Dashboards**...',
    '...rd Management\n\n<Check>\n  **Dashboards** as code are available in...',
    '... Tab:\n\n```python\nproject.**dashboard**.add_tab("Another Tab")\n`...'],
   'total_matches': 27},
  'filename': 'docs/platform/dashboard_add_panels.mdx'},
 {'title': 'Add dashboard panels (UI)',
  'description': 'How to design your Dashboard with custom Panels.',
  'content': {'matches': ['**Dashboards** let you create Panels to...',
    '...ject.\n\n<Check>\n  No-code **Dashboards** are available in the Evi...',
    '...anels appear on a single **Dashboard**. You can add multiple Ta...'],
   'total_matches': 11},
  'filename': 'docs/platform/dashboard_add_panels_ui.mdx'}]

In [20]:
import inspect

In [24]:
inspect.getdoc(evidently_search_tools.search

'Search index for results matching a query and optionally highlight them.\n\nArgs:\n    query: The search query to look up in index.\n\nReturns:\n    A list of search result objects.'

In [31]:
search_tool = {
    "type": "function",
    "name": evidently_search_tools.search.__name__,
    "description": inspect.getdoc(evidently_search_tools.search),
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query parameter"
            }
        },
        "required": [
            "query"
        ]
    }
}

get_file_tool = {
    "type": "function",
    "name": evidently_search_tools.get_file.__name__,
    "description": inspect.getdoc(evidently_search_tools.get_file),
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "filename parameter"
            }
        },
        "required": [
            "query"
        ]
    }
}

In [32]:
tool_pairs = [
    (evidently_search_tools.search, search_tool),
    (evidently_search_tools.get_file, get_file_tool),
]

In [39]:
tool_index = {d['name']: f for (f, d) in tool_pairs}
tools = [d for (_, d) in tool_pairs]

In [40]:
tools

[{'type': 'function',
  'name': 'search',
  'description': 'Search index for results matching a query and optionally highlight them.\n\nArgs:\n    query: The search query to look up in index.\n\nReturns:\n    A list of search result objects.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'query parameter'}},
   'required': ['query']}},
 {'type': 'function',
  'name': 'get_file',
  'description': "Retrieve a file's contents by filename.\n\nArgs:\n    filename: The filename of file to retrieve.\n\nReturns:\n    The file contents if found, otherwise an error message.",
  'parameters': {'type': 'object',
   'properties': {'filename': {'type': 'string',
     'description': 'filename parameter'}},
   'required': ['query']}}]

In [53]:
class Agent:

    def __init__(self, llm_client, model, instructions, tool_pairs):
        self.llm_client = llm_client
        self.model = model
        self.instructions = instructions
        self.tool_index = {d['name']: f for (f, d) in tool_pairs}
        self.tools = [d for (_, d) in tool_pairs]

    def make_call(self, tool_call):
        arguments = json.loads(tool_call.arguments)
        name = tool_call.name

        if name in self.tool_index:
            func = self.tool_index[name]
            result = func(**arguments)
        else:
            result = f'function {name} does not exist'

        return {
            "type": "function_call_output",
            "call_id": tool_call.call_id,
            "output": json.dumps(result),
        }   

    def loop(self, user_prompt, message_history=None):
        if not message_history:
            message_history = [
                {"role": "system", "content": self.instructions},
            ]
            
        message_history.append({"role": "user", "content": user_prompt})

        iteration_number = 0
    
        while True:
            response = self.llm_client.responses.create(
                model=self.model,
                input=message_history,
                tools=self.tools,
            )
        
            print(f'iteraration number {iteration_number}...') 
            message_history.extend(response.output)
        
            has_function_calls = False
        
            for message in response.output:
                if message.type == 'function_call':
                    print(f'executing {message.name}({message.arguments})...')
                    tool_call_output = self.make_call(message)
                    message_history.append(tool_call_output)
                    has_function_calls = True
        
                if message.type == 'message':
                    text = message.content[0].text
                    print('ASSISTANT:', text)
        
            iteration_number = iteration_number + 1
            print()
            
            if not has_function_calls:
                break

        return message_history        

    def qna(self):
        message_history = []

        while True:
            user_prompt = input('You:')
            if user_prompt.lower().strip() == 'stop':
                break
            
            message_history = self.loop(user_prompt, message_history)

        return message_history

In [54]:
instructions = """
You're a documentation assistant.

Answer the user question using only the documentation knowledge base.

Make 3 iterations:

1) First iteration:
   - Perform one search using the search tool to identify potentially relevant documents.
   - Explain (in 2–3 sentences) why this search query is appropriate for the user question.

2) Second iteration:
   - Analyze the results from the previous search.
   - Based on the filenames or documents returned, perform:
       - Up to 2 additional search queries to refine or expand coverage, and
       - One or more get_file calls to retrieve the full content of the most relevant documents.
   - For each search or get_file call, explain (in 2–3 sentences) why this action is necessary and how it helps answer the question.

3) Third iteration:
   - Analyze the retrieved document contents from get_file.
   - Synthesize the information from these documents into a final answer to the user.

IMPORTANT:
- At every step, explicitly explain your reasoning for each search query or file retrieval.
- Use only facts found in the documentation knowledge base.
- Do not introduce outside knowledge or assumptions.
- If the answer cannot be found in the retrieved documents, clearly inform the user.

Additional notes:
- The knowledge base is entirely about Evidently, so you do not need to include the word "evidently" in search queries.
- Prefer retrieving and analyzing full documents (via get_file) before producing the final answer.
"""

In [55]:
agent = Agent(
    llm_client=OpenAI(),
    model='gpt-4o-mini',
    instructions=instructions,
    tool_pairs=tool_pairs
)

In [52]:
messages = agent.loop('evidently adshboards') 

iteraration number 0...
executing search({"query":"dashboards"})...

iteraration number 1...
ASSISTANT: The search for "dashboards" yielded several documents that seem relevant to your inquiry. This query is appropriate as it directly targets the subject of dashboards, which appears to be your focus. The documents can help provide insights into how dashboards are structured, their purpose, and perhaps guide how to effectively use or create them.

Now, I will examine these results to identify the most pertinent documents and plan for deeper exploration through additional searches and document retrievals. 

Let's proceed with retrieving files that seem to provide foundational information about dashboards. Specifically, I will obtain the "Overview" document, which should provide a comprehensive introduction to dashboards. This foundational knowledge is essential before delving into more technical aspects of dashboard management or design.
executing get_file({"filename":"docs/platform/dash

In [56]:
agent.qna()

You: dashborads evidently 


iteraration number 0...
executing search({"query":"dashboards"})...

iteraration number 1...
ASSISTANT: The initial search for "dashboards" returned several relevant documents, indicating a focus on dashboard design, management, and panel types. This search is appropriate because it targets the specific functionality and features related to dashboards, which seems to match the user's inquiry. 

The results include guides on adding dashboard panels, an overview of dashboards, and details on panel types, making it likely that I can find useful information for your question about dashboards. 

Now, I will proceed to retrieve the content of the "Overview" document about dashboards as it seems fundamental, and I might also look for more details on adding panels since that appears to be a key function related to dashboards.
executing get_file({"filename":"docs/platform/dashboard_overview.mdx"})...
executing get_file({"filename":"docs/platform/dashboard_add_panels.mdx"})...

iteraration numbe

You: i need specific details


iteraration number 0...
executing search({"query":"dashboard panel types"})...
executing search({"query":"dashboard customization"})...

iteraration number 1...
executing get_file({"filename":"docs/platform/dashboard_panel_types.mdx"})...
executing get_file({"filename":"docs/platform/dashboard_add_panels_ui.mdx"})...

iteraration number 2...
ASSISTANT: Here are the specific details regarding dashboard customization and the different types of panels you can create:

### 1. **Adding Tabs**
- **Purpose:** Organize panels within the dashboard.
- **Steps to Add a Tab:**
  1. Enter "Edit" mode on the dashboard.
  2. Click the plus sign to add a new tab.
  3. Optionally create a custom tab by selecting "empty" and entering a name.

- **Pre-built Tabs:** You can start with dashboard templates that come with preset panel combinations, relevant to specific metrics. However, panels will remain empty until appropriate reports are added.

### 2. **Adding Panels**
- **Purpose:** Panels visualize eva

You: stop


[{'role': 'system',
  'content': '\nYou\'re a documentation assistant.\n\nAnswer the user question using only the documentation knowledge base.\n\nMake 3 iterations:\n\n1) First iteration:\n   - Perform one search using the search tool to identify potentially relevant documents.\n   - Explain (in 2–3 sentences) why this search query is appropriate for the user question.\n\n2) Second iteration:\n   - Analyze the results from the previous search.\n   - Based on the filenames or documents returned, perform:\n       - Up to 2 additional search queries to refine or expand coverage, and\n       - One or more get_file calls to retrieve the full content of the most relevant documents.\n   - For each search or get_file call, explain (in 2–3 sentences) why this action is necessary and how it helps answer the question.\n\n3) Third iteration:\n   - Analyze the retrieved document contents from get_file.\n   - Synthesize the information from these documents into a final answer to the user.\n\nIMPORT

In [65]:
from agent_lib import AbstractAgent

In [67]:
class OpenAIResponsesAgent(AbstractAgent):

    def format_tool_call_output(self, tool_call_id, tool_call_output):
        return {
            "type": "function_call_output",
            "call_id": tool_call_id,
            "output": tool_call_output,
        }  

    def loop(self, user_prompt, message_history=None):
        if not message_history:
            message_history = [
                {"role": "system", "content": self.instructions},
            ]
            
        message_history.append({"role": "user", "content": user_prompt})

        iteration_number = 0
    
        while True:
            response = self.llm_client.responses.create(
                model=self.model,
                input=message_history,
                tools=self.tools,
            )

            print(f'iteraration number {iteration_number}...') 
            message_history.extend(response.output)
        
            has_function_calls = False
        
            for message in response.output:
                if message.type == 'function_call':
                    print(f'executing {message.name}({message.arguments})...')
                    tool_call_output = self.make_call(message)
                    message_history.append(tool_call_output)
                    has_function_calls = True
        
                if message.type == 'message':
                    text = message.content[0].text
                    print('ASSISTANT:', text)
        
            iteration_number = iteration_number + 1
            print()
            
            if not has_function_calls:
                break

        return message_history         

    

In [68]:

agent = OpenAIResponsesAgent(
    llm_client=OpenAI(),
    model='gpt-4o-mini',
    instructions=instructions,
    tool_pairs=tool_pairs
)
 

In [69]:
messages = agent.loop('evidently adshboards') 

iteraration number 0...
executing search({"query":"dashboards"})...

iteraration number 1...
ASSISTANT: The search for "dashboards" is appropriate because it targets the core functionality of creating and managing dashboards, which seems to be the user's area of interest. It has returned relevant documents that could provide insights into dashboard management, including options for customization and types of panels available.

Now, I will proceed to analyze the results to refine the search and gather necessary details. The documents titled "Add dashboard panels (API)" and "Overview" are particularly relevant. I will retrieve their full content to better understand how to create and manage dashboards effectively.
executing get_file({"filename":"docs/platform/dashboard_add_panels.mdx"})...
executing get_file({"filename":"docs/platform/dashboard_overview.mdx"})...

iteraration number 2...
ASSISTANT: The retrieved documents offer a comprehensive understanding of dashboards:

1. **"Add dash